###Step1. Define ImageDataset

In [1]:
import glob #파일들의 리스트 추출 모듈
import random
import os

from torch.utils.data import Dataset, DataLoader #
from PIL import Image
import torchvision.transforms as transforms #
import sys #
import torch.nn as nn #
import torch.nn.functional as F #
import torch #

import os
import numpy as np
import math
import itertools
import datetime
import time

from torchvision.utils import save_image, make_grid
from torchvision import datasets
from torch.autograd import Variable

In [2]:
def to_rgb(image):
  rgb_image = Image.new("RGB", image.size)
  rgb_image.paste(image)
  return rgb_image

In [ ]:
class ImageDataset(Dataset):
  def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
    self.transform = transforms.Compose(transforms_)
    self.unaligned = unaligned
    
    if mode=="train":
      self.files_A = sorted(glob.glob(os.path.join(root, "trainA") + "/*.*"))
      self.files_B = sorted(glob.glob(os.path.join(root, "trainB") + "/*.*"))
    else:
      self.files_A = sorted(glob.glob(os.path.join(root, "testA") + "/*.*"))
      self.files_B = sorted(glob.glob(os.path.join(root, "testB") + "/*.*"))

  def __getitem__(self, index):
    image_A = Image.open(self.files_A[index % len(self.files_A)])

    if self.unaligned: #학습할 쌍 무작위 선택 가능
      image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) -1)])
    else:
      image_B = Image.open(self.files_B[index % len(self.files_B)])

    if image_A.mode != "RGB":
      image_A = to_rgb(image_A)
    if image_B.mode != "RGB":
      image_B = to_rgb(image_B)   

    item_A = self.transform(image_A) #transform 함수를 통해 torch의 tensor 형태로 변환
    item_B = self.transform(image_B)

    return {"A": item_A, "B": item_B}

  def __len__(self):
      return max(len(self.files_A), len(self.files_B))   

###Step2. Define the transform function

In [ ]:
transforms_ = [
    transforms.Resize(int(img_height*1.12), Image.BICUBIC),   #1. PIL 이미지를 resize, 보간 방식은 BICUBIC
    transforms.RandomCrop((img_height, img_width)),           #PIL 이미지를 무작위로 crop
    transforms.RandomHorizontalFlip(),                        #PIL 이미지를 무작위로 좌우 변환
    transforms.ToTensor(),                                   #2. to_tensor: 0~1 사이의 Tensor 자료형으로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5,)), #3. normalize: RGB 채널별로 평균 0.5, 표준편차 0.5로 정규화
]

###Step3. Define the DataLoader

In [ ]:
#Training data loader
dataloader= DataLoader(
    ImageDataset("./dataset/%s" % dataset_name, transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

#Test data loader
val_dataloader= DataLoader(
    ImageDataset("./dataset/%s" % dataset_name, transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)